### Automatic optimization strategies
In this code document, we will be experimenting with algorithms that can identify optimization routes in the production of 7-dehydrocholesterol in yeast.

In [2]:
from cobra.io import read_sbml_model, write_sbml_model
from cobra import Reaction, Metabolite

In [3]:
model = read_sbml_model('data/yeastGEM_het.xml')

model

Name,M_yeastGEM_v8__46__4__46__2
Memory address,0x02ad8585e070
Number of metabolites,2745
Number of reactions,4062
Number of groups,90
Objective expression,1.0*r_2111 - 1.0*r_2111_reverse_58b69
Compartments,"cell envelope, cytoplasm, extracellular, mitochondrion, nucleus, peroxisome, endoplasmic reticulum, Golgi, lipid particle, vacuole, endoplasmic reticulum membrane, vacuolar membrane, Golgi membrane, mitochondrial membrane"


### Automatic optimization using OptGene

Note: This section of the code is heavily based around the notebook: 09-Predict-gene-knockout-strategies and this notebook: http://cameo.bio/05-predict-gene-knockout-strategies.html

The OptGene algorithm can be used to find genes which can be knocked out or mutated to increase the production of our target compound. The algorithm uses an approach, which takes a very long time to run. 
To make the program a tiny bit faster, all the essential reactions can be found and excluded from analysis, as they should not be knocked out in any case. Knocking out essential genes would stop growth.

In [87]:
# In this block, we are finding a list of the essential reactions and the non-essential reactions. 
# By looping over all reactions in the model and seeing whether they can grow, we can sort them into the two different lists

#%%time

essential = []
non_essential = []

for reaction in model.reactions:
    with model:
        reaction.knock_out()
        if model.slim_optimize(error_value=0.) == 0.0:
            essential.append(reaction)
        else:
            non_essential.append(reaction)

In [90]:
# Following this, we can find the genes associated with all of the essential reactions

#%%time
ess = []
for i in range (1,len(essential)):
    for gene in list(essential[i].genes):
        ess.append(gene)

In [ ]:
from cameo.strain_design.heuristic.evolutionary_based import OptGene

In [52]:
from cameo.strain_design import OptGene
optgene = OptGene(model)
result = optgene.run(target=model.reactions.R07215,
                     biomass=model.reactions.r_2111,
                     substrate=model.reactions.r_1714,
                     max_evaluations=5000,
                     plot=False)

Starting optimization at Tue, 24 Nov 2020 18:32:19


HBox()

Finished after 02:55:50


In [58]:
result

,reactions,genes,size,fva_min,fva_max,target_flux,biomass_flux,yield,fitness


As we can be seen from this, we have no optimization targets for the OptGene analysis

### OptKnock

Similar to OptGene, OptKnock is an algorithm that can produce knock-out strains to increase the target production. 

In [4]:
from cameo.strain_design.deterministic.linear_programming import OptKnock
from cameo.strain_design import OptKnock

In [5]:
optknock = OptKnock(model, fraction_of_optimum=0.1) #excluded reactions virker ikke

C:\Users\jmejl\AppData\Roaming\Python\Python38\site-packages\cameo\strain_design\deterministic\linear_programming.py:118: UserWarning: You are trying to run OptKnock with glpk_interface. This might not end well.
  warnings.warn("You are trying to run OptKnock with %s. This might not end well." %
C:\Users\jmejl\anaconda3\lib\site-packages\cobra\core\group.py:109: UserWarning: need to pass in a list
  warn("need to pass in a list")


Infeasible: None (infeasible).

In [ ]:
result = optknock.run(max_knockouts=3, target="R07215", biomass="r_2111")

In [ ]:
result

In [ ]:
result.plot(plotter, height=400)